In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
data_path="../input/ieee-fraud-detection/"
print(os.listdir(data_path))

['test_transaction.csv', 'test_identity.csv', 'train_transaction.csv', 'sample_submission.csv', 'train_identity.csv']


In [2]:
from sklearn import preprocessing
import xgboost as xgb

In [3]:
train_transaction = pd.read_csv(data_path+'train_transaction.csv', index_col='TransactionID', nrows=3000)
test_transaction = pd.read_csv(data_path+'test_transaction.csv', index_col='TransactionID',nrows=1000)

train_identity = pd.read_csv(data_path+'train_identity.csv', index_col='TransactionID', nrows=3000)
test_identity = pd.read_csv(data_path+'test_identity.csv', index_col='TransactionID',nrows=1000)

sample_submission = pd.read_csv(data_path+'sample_submission.csv', index_col='TransactionID')

In [4]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

(3000, 433)
(1000, 432)


In [5]:
del train, test, train_transaction, train_identity, test_transaction, test_identity

In [6]:
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

In [7]:
clf = xgb.XGBClassifier(n_estimators=500,
                        n_jobs=4,
                        max_depth=9,
                        learning_rate=0.05,
                        subsample=0.9,
                        colsample_bytree=0.9,
                        missing=-999)

clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=-999, n_estimators=500, n_jobs=4,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

In [ ]:
sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1]
sample_submission.to_csv('simple_xgboost.csv')